# Retrieval Augmented Generation

## Setup API clients

In [1]:
import os
import dotenv
import openai
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.core.credentials import AzureKeyCredential

dotenv.load_dotenv()

# Initialize Azure OpenAI client
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")

openai_client = openai.AzureOpenAI(
    api_version="2023-07-01-preview",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com")

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding

# Initialize Azure search client
AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_ENDPOINT = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"
AZURE_SEARCH_SERVICE_KEY = os.getenv("AZURE_SEARCH_SERVICE_KEY")
search_service_cred = AzureKeyCredential(AZURE_SEARCH_SERVICE_KEY)

AZURE_SEARCH_FULL_INDEX = "gptkbindex"
search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=search_service_cred)


## Tìm kiếm với Semantic và Hybrid search

The search call below does a **hybrid search**, performing both a full-text search and a vector search in parallel.
It merges those results using Reciprocal Rank Fusion (RRF). 
Finally, it re-ranks the merged results using the AI Search semantic ranker, a re-ranking model.

Trước tiên chúng ta cần thiết lập semantic search cho index

In [ ]:

from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SemanticConfiguration, SemanticPrioritizedFields, SemanticField, SemanticSearch

index_client = SearchIndexClient(endpoint=AZURE_SEARCH_ENDPOINT, credential=search_service_cred)
semantic_config = SemanticConfiguration(
                    name="default-semantic-config",
                    prioritized_fields=SemanticPrioritizedFields(
                        content_fields=[SemanticField(field_name="content")]
                    )
                )

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])     
current_index = index_client.get_index(AZURE_SEARCH_FULL_INDEX)
current_index.semantic_search = semantic_search
index_client.create_or_update_index(current_index)

## Chuẩn bị câu hỏi

In [3]:
user_question = "máy tính bảng có khả năng chơi game"

In [ ]:
def search(user_question):
        user_question_vector = get_embedding(user_question)
        r = search_client.search(
                user_question,
                top=5, 
                vector_queries=[
                        VectorizedQuery(vector=user_question_vector, k_nearest_neighbors=50, fields="embedding")],
                query_type="semantic",
                semantic_configuration_name="default-semantic-config")

        sources = "\n".join([f"{doc['sourcefile']}: {doc['content']}\n" for doc in r])
        return sources

result = search(user_question)
print(result)

## Gửi câu hỏi và thông tin sản phẩm tới LLM

In [ ]:
SYSTEM_MESSAGE = """
Assistant helps answer customer's questions about the available products. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below.
Each source has a name followed by colon and the actual information, include the source name for each fact you use.
Use square brackets to reference the source, for example [info1.txt].
"""
user_question = "có máy in màu không?"

sources = search(user_question)

messages = [
    {"role": "system", "content": SYSTEM_MESSAGE},
    {"role": "user", "content": user_question + "\nSources: " + sources}
]
# Now we can use the matches to generate a response
response = openai_client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    temperature=0.7,
    messages=messages
)

answer = response.choices[0].message.content
print("Câu hỏi: ", user_question)
print("Bot: ", answer)


### Cải thiện để xử lý conversation

In [25]:
def generate_query(messages, question):
    clone_messages = list(messages)[1:]
    clone_messages.append({"role": "user", 
                           "content": f"Đây là câu hỏi của user: {question}. Dựa trên lịch sử và câu hỏi hiện tại, tạo query tối đa 10 từ để tìm kiếm. Đừng đưa ra gợi ý ngoài câu hỏi."})
    response = openai_client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        temperature=0,
        messages=clone_messages
    )

    return response.choices[0].message.content


In [ ]:
from time import sleep


SYSTEM_MESSAGE = """
Assistant helps answer customer's questions about the available products. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below.
Each source has a name followed by colon and the actual information, include the source name for each fact you use.
Use square brackets to reference the source, for example [info1.txt].
"""


messages = [{"role": "system", "content": SYSTEM_MESSAGE}]

while(True):
    sleep(1)
    question = input()
    if question == "exit":
        break
    
    print("Human > " + question)
    query = generate_query(messages=messages, question=question)
    print(f"\tGenerated query: {query}")

    sources = search(query)
    
    USER_MESSAGE = question + "\nSources: " + sources
    messages.append( {"role": "user", "content": USER_MESSAGE})
    
    # Now we can use the matches to generate a response
    response = openai_client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        temperature=0,
        messages=messages
    )

    answer = response.choices[0].message.content
    messages.append({"role": "assistant", "content": answer})
    print("Bot > " + answer)
